In [ ]:
import proptech
import config as cfg

In [ ]:
# Remember to enter some data into the config file

appid = cfg.config['appid']
secret = cfg.config['secret']
owner = cfg.config['owner']

connection = proptech.ProptechConnection(appid, secret, owner)


In [ ]:
# Try using some Panda Dataframes!
import pandas as pd
from IPython.display import display, HTML

plt.rcParams["figure.figsize"] = (10, 8) # (w, h)

# Fetch all the quantity kinds used by ProptechOS and display them as an HTML table
kinds = connection.fetch('quantitykind')
df = pd.DataFrame(kinds)

#display(HTML(df.to_html()))
df.head()

In [ ]:
# Find some sensors - also add the description to sensors... but picking it up from the source
sensors = connection.fetch('sensor', 400)
df = pd.DataFrame(sensors['content'])
df["description"] = ""
#display(HTML(df.to_html()))

# Add the description into the row
for index, sensor in df.iterrows():
    if 'description' in sensor['source']:
        sensor['description'] = sensor['source']['description']

# Skip a bunch of data
df1 = df[['littera', 'id', 'description', 'deviceMeasurementUnit', 'deviceQuantityKind', 'source']]

df1.head()
#display(HTML(df1.to_html()))



In [ ]:

print(1)


In [ ]:
connection.plot_tag('VP2_EP14_PV1')

In [ ]:
connection.plot_tag('VP2_EP14_PV2')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# Pick a few sensors from the above list and plot
# This is by index
id = df1.iloc[7]['id']

# To select rows whose column value equals a specific value (can selection of id2 be made simpler???)
row = df.loc[df['littera'] == 'KV2_GT41_PV']
id2 = row['id'].values[0]

print(id)
print(id2)

data = connection.fetch_data(id)
df2 = pd.DataFrame(data)
df2.plot()

# Configure the plot a bit - with ylabel and title
plt.figure()
data = connection.fetch_data(id2)
df3 = pd.DataFrame(data)
ax = df3.plot(title = row['littera'].values[0] + "  " + row['description'].values[0])
ax.set_xlabel('time')
ax.set_ylabel(row['deviceMeasurementUnit'].values[0])
plt.legend(loc='best')


## SCADA - Ventilation system
![](LB04-air.png)


In [ ]:
# Select all rows that contains LB04 and GT11 from the sensors
lb04= df1[df1['littera'].str.contains('LB04', regex=False)]
gt41 = lb04[lb04['littera'].str.contains('GT41', regex=False)]
gt44 = lb04[lb04['littera'].str.contains('GT44', regex=False)]

# LB04 = Air handling unit / Luft Behandling 04

# Print all of it
display(HTML(gt44.to_html()))

In [ ]:
# Create your own plot function in the notebook - loads two dataseries based on "rows" / sensors
def plot_large(title, row1, row2):
    # Load data, create dataframes, and replace names of series
    data1 = connection.fetch_data(row1['id'].values[0])
    data2 = connection.fetch_data(row2['id'].values[0])
    df1 = pd.DataFrame(data1)
    df2 = pd.DataFrame(data2)

    df1['observationTime'] = pd.to_datetime(df1['observationTime'], format="%Y-%m-%dT%H:%M:%SZ")
    df2['observationTime'] = pd.to_datetime(df2['observationTime'], format="%Y-%m-%dT%H:%M:%SZ")
    fig, ax = plt.subplots()
    fig.autofmt_xdate()
    formatter = mdates.DateFormatter("%Y-%m-%d %H:%M")
    ax.xaxis.set_major_formatter(formatter)
    plt.plot(df1['observationTime'], df1['value'], label=row1['littera'].values[0])
    plt.plot(df2['observationTime'], df2['value'], label=row2['littera'].values[0])

    #ax = df1.plot()
    plt.xlabel('timestamp')
    plt.ylabel(row1['deviceMeasurementUnit'].values[0])
    # Plot the next one in the same graph
    #df2.plot(ax = ax, title = title)
    plt.legend(loc='best')

#connection.plot_tag('LB04_GT44_PV')
    
gt41_pv = df1[df1['littera'] == 'LB04_GT41_PV']
gt44_pv = df1[df1['littera'] == 'LB04_GT44_PV']

plot_large("Ventilationstemp", gt41_pv, gt44_pv)


## SCADA - control system image - heating system

![](VP2-KB2-undercentral.png)



## SCADA - Heating system 2

![](VS2-VV2-undercentral.png)

